In [51]:
import tensorflow as tf
import numpy as np
tf.set_random_seed(777)  # for reproducibility

from sklearn.datasets import load_iris
iris_dataset = load_iris()

print("iris_dataset의 키: \n{}".format(iris_dataset.keys()))
print(iris_dataset['DESCR'][:193] + "\n...")
print("타깃의 이름: {}".format(iris_dataset['target_names']))
print("특성의 이름: \n{}".format(iris_dataset['feature_names']))
print("data의 타입: {}".format(type(iris_dataset['data'])))
print("data의 크기: {}".format(iris_dataset['data'].shape))
print("data의 처음 다섯 행:\n{}".format(iris_dataset['data'][:5]))
print("target의 타입: {}".format(type(iris_dataset['target'])))
print("target의 크기: {}".format(iris_dataset['target'].shape))
print("타깃:\n{}".format(iris_dataset['target']))

iris_dataset의 키: 
dict_keys(['data', 'target', 'target_names', 'DESCR', 'feature_names', 'filename'])
.. _iris_dataset:

Iris plants dataset
--------------------

**Data Set Characteristics:**

    :Number of Instances: 150 (50 in each of three classes)
    :Number of Attributes: 4 numeric, pre
...
타깃의 이름: ['setosa' 'versicolor' 'virginica']
특성의 이름: 
['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']
data의 타입: <class 'numpy.ndarray'>
data의 크기: (150, 4)
data의 처음 다섯 행:
[[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]]
target의 타입: <class 'numpy.ndarray'>
target의 크기: (150,)
타깃:
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]


In [52]:
x = np.float32(iris_dataset['data'])
y = np.reshape(iris_dataset['target'],(150,1))

print(x.shape, y.shape)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, random_state=0)

print("X_train 크기: {}".format(X_train.shape))
print("y_train 크기: {}".format(y_train.shape))

print("X_test 크기: {}".format(X_test.shape))
print("y_test 크기: {}".format(y_test.shape))


(150, 4) (150, 1)
X_train 크기: (112, 4)
y_train 크기: (112, 1)
X_test 크기: (38, 4)
y_test 크기: (38, 1)


In [53]:
nb_classes = 3  

X = tf.placeholder(tf.float32, [None, 4])
Y = tf.placeholder(tf.int64, [None, 1])

Y_one_hot = tf.one_hot(Y, nb_classes)  # one hot
print("one_hot:", Y_one_hot)
Y_one_hot = tf.reshape(Y_one_hot, [-1, nb_classes])
print("reshape one_hot:", Y_one_hot)

one_hot: Tensor("one_hot_9:0", shape=(?, 1, 3), dtype=float32)
reshape one_hot: Tensor("Reshape_7:0", shape=(?, 3), dtype=float32)


In [68]:
W = tf.Variable(tf.random_normal([4, nb_classes]), name='weight')
b = tf.Variable(tf.random_normal([nb_classes]), name='bias')

# tf.nn.softmax computes softmax activations
# softmax = exp(logits) / reduce_sum(exp(logits), dim)
logits = tf.matmul(X, W) + b
hypothesis = tf.nn.softmax(logits)

# Cross entropy cost/loss
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits,
                                                                 labels=tf.stop_gradient([Y_one_hot])))
train = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

# Test model & accuracy
prediction = tf.argmax(hypothesis, 1)
correct_prediction = tf.equal(prediction, tf.argmax(Y_one_hot, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# parameters
num_epochs = 30
batch_size = 40
num_iterations = int(112 / batch_size)

In [69]:
# Launch graph
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    #Training cycle
    for epoch in range(num_epochs):
        avg_cost = 0

        for i in range(num_iterations):
            _, cost_val = sess.run([train, cost], feed_dict={X: X_train, Y: y_train})
            avg_cost += cost_val / num_iterations

        print("Epoch: {:04d}, Cost: {:.3f}\tAcc: {:.2%}".format(epoch + 1, avg_cost, acc_val))

    print("Learning finished")
    
    # Test the model using test sets
    pred = sess.run(prediction, feed_dict={X: X_test})
    # y_data: (N,1) = flatten => (N, ) matches pred.shape
    for p, y in zip(pred, y_test.flatten()):
        print("[{}] Prediction: {} True Y: {}".format(p == int(y), p, int(y)))
    print("Accuracy: ",accuracy.eval(session=sess, feed_dict={X: X_test, Y: y_test}),)
        

Epoch: 0001, Cost: 6.757	Acc: 85.71%
Epoch: 0002, Cost: 4.474	Acc: 85.71%
Epoch: 0003, Cost: 3.470	Acc: 85.71%
Epoch: 0004, Cost: 3.217	Acc: 85.71%
Epoch: 0005, Cost: 2.987	Acc: 85.71%
Epoch: 0006, Cost: 2.765	Acc: 85.71%
Epoch: 0007, Cost: 2.543	Acc: 85.71%
Epoch: 0008, Cost: 2.320	Acc: 85.71%
Epoch: 0009, Cost: 2.098	Acc: 85.71%
Epoch: 0010, Cost: 1.882	Acc: 85.71%
Epoch: 0011, Cost: 1.679	Acc: 85.71%
Epoch: 0012, Cost: 1.494	Acc: 85.71%
Epoch: 0013, Cost: 1.330	Acc: 85.71%
Epoch: 0014, Cost: 1.190	Acc: 85.71%
Epoch: 0015, Cost: 1.076	Acc: 85.71%
Epoch: 0016, Cost: 0.988	Acc: 85.71%
Epoch: 0017, Cost: 0.920	Acc: 85.71%
Epoch: 0018, Cost: 0.870	Acc: 85.71%
Epoch: 0019, Cost: 0.833	Acc: 85.71%
Epoch: 0020, Cost: 0.804	Acc: 85.71%
Epoch: 0021, Cost: 0.781	Acc: 85.71%
Epoch: 0022, Cost: 0.763	Acc: 85.71%
Epoch: 0023, Cost: 0.747	Acc: 85.71%
Epoch: 0024, Cost: 0.733	Acc: 85.71%
Epoch: 0025, Cost: 0.721	Acc: 85.71%
Epoch: 0026, Cost: 0.710	Acc: 85.71%
Epoch: 0027, Cost: 0.699	Acc: 85.71%
E